In [2]:
ls

Banking Marketing Targets.ipynb  train.csv
test.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn.preprocessing import StandardScaler

In [3]:
df_raw = pd.read_csv('train.csv', delimiter=';', encoding='UTF-8')
df_raw.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [4]:
df_raw.describe(include = 'all')

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
count,45211.000000,45211,45211,45211,45211,45211.000000,45211,45211,45211,45211.000000,45211,45211.000000,45211.000000,45211.000000,45211.000000,45211,45211
unique,NaN,12,3,4,2,NaN,2,2,3,NaN,12,NaN,NaN,NaN,NaN,4,2
top,NaN,blue-collar,married,secondary,no,NaN,yes,no,cellular,NaN,may,NaN,NaN,NaN,NaN,unknown,no
freq,NaN,9732,27214,23202,44396,NaN,25130,37967,29285,NaN,13766,NaN,NaN,NaN,NaN,36959,39922
mean,40.936210,NaN,NaN,NaN,NaN,1362.272058,NaN,NaN,NaN,15.806419,NaN,258.163080,2.763841,40.197828,0.580323,NaN,NaN
std,10.618762,NaN,NaN,NaN,NaN,3044.765829,NaN,NaN,NaN,8.322476,NaN,257.527812,3.098021,100.128746,2.303441,NaN,NaN
min,18.000000,NaN,NaN,NaN,NaN,-8019.000000,NaN,NaN,NaN,1.000000,NaN,0.000000,1.000000,-1.000000,0.000000,NaN,NaN
25%,33.000000,NaN,NaN,NaN,NaN,72.000000,NaN,NaN,NaN,8.000000,NaN,103.000000,1.000000,-1.000000,0.000000,NaN,NaN
50%,39.000000,NaN,NaN,NaN,NaN,448.000000,NaN,NaN,NaN,16.000000,NaN,180.000000,2.000000,-1.000000,0.000000,NaN,NaN
75%,48.000000,NaN,NaN,NaN,NaN,1428.000000,NaN,NaN,NaN,21.000000,NaN,319.000000,3.000000,-1.000000,0.000000,NaN,NaN


Results from EDA above:
Numeric Data:
1) Age range from 18 - 95, mean = 40, std = 10.6
2) Balance - 1362 mean, std = 3044
Binary Data:
1) Default history - Yes / No
2) Housing loan - Yes / No
3) Personal Loan - Yes / No
Categorical Data:
1) Job - 12 categories
2) Marital - 3 categories
3) Education - 4 categories

9) contact: contact communication type (categorical: "unknown","telephone","cellular")
10) day: last contact day of the month (numeric)
11) month: last contact month of year (categorical: "jan", "feb", "mar", …, "nov", "dec")
12) duration: last contact duration, in seconds (numeric)
13) campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
14) pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric, -1 means client was not previously contacted)
15) previous: number of contacts performed before this campaign and for this client (numeric)
16) poutcome: outcome of the previous marketing campaign (categorical: "unknown","other","failure","success")

In [5]:
df_raw.dtypes

age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
y            object
dtype: object

Binary variables to be mapped to 0 & 1 respectively

1) Default
2) Housing
3) Loan

In [6]:
df_raw['default'].value_counts()

no     44396
yes      815
Name: default, dtype: int64

In [7]:
df_raw['default'].replace({'no':0, 'yes':1},inplace=True)

In [8]:
df_raw['default'].value_counts()

0    44396
1      815
Name: default, dtype: int64

In [9]:
df_raw['housing'].value_counts()

yes    25130
no     20081
Name: housing, dtype: int64

In [10]:
df_raw['housing'].replace({'yes': 1, 'no': 0}, inplace=True)

In [11]:
df_raw['housing'].value_counts()

1    25130
0    20081
Name: housing, dtype: int64

In [12]:
df_raw['loan'].value_counts()

no     37967
yes     7244
Name: loan, dtype: int64

In [13]:
df_raw['loan'].replace({'no':0, 'yes':1},inplace=True)

In [14]:
df_raw['loan'].value_counts()

0    37967
1     7244
Name: loan, dtype: int64

In [15]:
df_raw['job'].value_counts()

blue-collar      9732
management       9458
technician       7597
admin.           5171
services         4154
retired          2264
self-employed    1579
entrepreneur     1487
unemployed       1303
housemaid        1240
student           938
unknown           288
Name: job, dtype: int64

In [16]:
df_work = df_raw.copy()

In [17]:
df_work.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,0,2143,1,0,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,0,29,1,0,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,0,2,1,1,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,0,1506,1,0,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,0,1,0,0,unknown,5,may,198,1,-1,0,unknown,no


Note that month column has not been converted to dummy values

In [18]:
df_dummy = pd.get_dummies(df_work, columns=['job','marital','education','contact','poutcome'], drop_first=True)

In [19]:
from sklearn.preprocessing import StandardScaler

In [20]:
scaler = StandardScaler()

In [21]:
df_standardized = df_dummy.copy()

In [22]:
df_standardized['age'] = scaler.fit(df_standardized[['age']]).transform(df_standardized[['age']])

In [23]:
df_standardized.head()

,age,default,balance,housing,loan,day,month,duration,campaign,pdays,...,marital_married,marital_single,education_secondary,education_tertiary,education_unknown,contact_telephone,contact_unknown,poutcome_other,poutcome_success,poutcome_unknown
0,1.606965,0,2143,1,0,5,may,261,1,-1,...,1,0,0,1,0,0,1,0,0,1
1,0.288529,0,29,1,0,5,may,151,1,-1,...,0,1,1,0,0,0,1,0,0,1
2,-0.747384,0,2,1,1,5,may,76,1,-1,...,1,0,1,0,0,0,1,0,0,1
3,0.571051,0,1506,1,0,5,may,92,1,-1,...,1,0,0,0,1,0,1,0,0,1
4,-0.747384,0,1,0,0,5,may,198,1,-1,...,0,1,0,0,1,0,1,0,0,1


In [24]:
df_standardized['balance'] = scaler.fit(df_standardized[['balance']]).transform(df_standardized[['balance']])

In [25]:
df_standardized.head()

,age,default,balance,housing,loan,day,month,duration,campaign,pdays,...,marital_married,marital_single,education_secondary,education_tertiary,education_unknown,contact_telephone,contact_unknown,poutcome_other,poutcome_success,poutcome_unknown
0,1.606965,0,0.256419,1,0,5,may,261,1,-1,...,1,0,0,1,0,0,1,0,0,1
1,0.288529,0,-0.437895,1,0,5,may,151,1,-1,...,0,1,1,0,0,0,1,0,0,1
2,-0.747384,0,-0.446762,1,1,5,may,76,1,-1,...,1,0,1,0,0,0,1,0,0,1
3,0.571051,0,0.047205,1,0,5,may,92,1,-1,...,1,0,0,0,1,0,1,0,0,1
4,-0.747384,0,-0.447091,0,0,5,may,198,1,-1,...,0,1,0,0,1,0,1,0,0,1


In [26]:
df_standardized['duration'] = scaler.fit(df_standardized[['duration']]).transform(df_standardized[['duration']])

In [27]:
df_standardized.head()

,age,default,balance,housing,loan,day,month,duration,campaign,pdays,...,marital_married,marital_single,education_secondary,education_tertiary,education_unknown,contact_telephone,contact_unknown,poutcome_other,poutcome_success,poutcome_unknown
0,1.606965,0,0.256419,1,0,5,may,0.011016,1,-1,...,1,0,0,1,0,0,1,0,0,1
1,0.288529,0,-0.437895,1,0,5,may,-0.416127,1,-1,...,0,1,1,0,0,0,1,0,0,1
2,-0.747384,0,-0.446762,1,1,5,may,-0.707361,1,-1,...,1,0,1,0,0,0,1,0,0,1
3,0.571051,0,0.047205,1,0,5,may,-0.645231,1,-1,...,1,0,0,0,1,0,1,0,0,1
4,-0.747384,0,-0.447091,0,0,5,may,-0.233620,1,-1,...,0,1,0,0,1,0,1,0,0,1


In [29]:
df_standardized['campaign'] = scaler.fit(df_standardized[['campaign']]).transform(df_standardized[['campaign']])

In [31]:
df_standardized.head()

,age,default,balance,housing,loan,day,month,duration,campaign,pdays,...,marital_married,marital_single,education_secondary,education_tertiary,education_unknown,contact_telephone,contact_unknown,poutcome_other,poutcome_success,poutcome_unknown
0,1.606965,0,0.256419,1,0,5,may,0.011016,-0.569351,-1,...,1,0,0,1,0,0,1,0,0,1
1,0.288529,0,-0.437895,1,0,5,may,-0.416127,-0.569351,-1,...,0,1,1,0,0,0,1,0,0,1
2,-0.747384,0,-0.446762,1,1,5,may,-0.707361,-0.569351,-1,...,1,0,1,0,0,0,1,0,0,1
3,0.571051,0,0.047205,1,0,5,may,-0.645231,-0.569351,-1,...,1,0,0,0,1,0,1,0,0,1
4,-0.747384,0,-0.447091,0,0,5,may,-0.233620,-0.569351,-1,...,0,1,0,0,1,0,1,0,0,1


In [34]:
df_standardized.rename(columns={'campaign':'number of contacts'}, inplace=True)

In [35]:
df_standardized.head()

,age,default,balance,housing,loan,day,month,duration,number of contacts,pdays,...,marital_married,marital_single,education_secondary,education_tertiary,education_unknown,contact_telephone,contact_unknown,poutcome_other,poutcome_success,poutcome_unknown
0,1.606965,0,0.256419,1,0,5,may,0.011016,-0.569351,-1,...,1,0,0,1,0,0,1,0,0,1
1,0.288529,0,-0.437895,1,0,5,may,-0.416127,-0.569351,-1,...,0,1,1,0,0,0,1,0,0,1
2,-0.747384,0,-0.446762,1,1,5,may,-0.707361,-0.569351,-1,...,1,0,1,0,0,0,1,0,0,1
3,0.571051,0,0.047205,1,0,5,may,-0.645231,-0.569351,-1,...,1,0,0,0,1,0,1,0,0,1
4,-0.747384,0,-0.447091,0,0,5,may,-0.233620,-0.569351,-1,...,0,1,0,0,1,0,1,0,0,1


In [38]:
## Replacing the '-1' which meant not contacted to '0'
df_standardized['pdays'].replace({-1: 0}, inplace=True)

In [40]:
df_standardized['pdays'] = scaler.fit(df_standardized[['pdays']]).transform(df_standardized[['pdays']])

In [41]:
df_standardized['previous'] = scaler.fit(df_standardized[['previous']]).transform(df_standardized[['previous']])

In [42]:
df_standardized.rename(columns={'previous': 'previous # contacts'}, inplace=True)

In [43]:
df_standardized.head()

,age,default,balance,housing,loan,day,month,duration,number of contacts,pdays,...,marital_married,marital_single,education_secondary,education_tertiary,education_unknown,contact_telephone,contact_unknown,poutcome_other,poutcome_success,poutcome_unknown
0,1.606965,0,0.256419,1,0,5,may,0.011016,-0.569351,-0.411009,...,1,0,0,1,0,0,1,0,0,1
1,0.288529,0,-0.437895,1,0,5,may,-0.416127,-0.569351,-0.411009,...,0,1,1,0,0,0,1,0,0,1
2,-0.747384,0,-0.446762,1,1,5,may,-0.707361,-0.569351,-0.411009,...,1,0,1,0,0,0,1,0,0,1
3,0.571051,0,0.047205,1,0,5,may,-0.645231,-0.569351,-0.411009,...,1,0,0,0,1,0,1,0,0,1
4,-0.747384,0,-0.447091,0,0,5,may,-0.233620,-0.569351,-0.411009,...,0,1,0,0,1,0,1,0,0,1


In [44]:
df_standardized.rename(columns={'pdays': '#days gap b/w contact'}, inplace=True)

Initiate and populate the train variables i.e. x_train and y_train

In [49]:
columns_order = df_standardized.columns.to_list()

In [54]:
columns_order = ['age',
 'default',
 'balance',
 'housing',
 'loan',
 'day',
 'month',
 'duration',
 'number of contacts',
 '#days gap b/w contact',
 'previous # contacts',
 'job_blue-collar',
 'job_entrepreneur',
 'job_housemaid',
 'job_management',
 'job_retired',
 'job_self-employed',
 'job_services',
 'job_student',
 'job_technician',
 'job_unemployed',
 'job_unknown',
 'marital_married',
 'marital_single',
 'education_secondary',
 'education_tertiary',
 'education_unknown',
 'contact_telephone',
 'contact_unknown',
 'poutcome_other',
 'poutcome_success',
 'poutcome_unknown',
'y']

In [55]:
df_standardized = df_standardized[columns_order]

In [64]:
df_standardized.iloc[[,0:32]]

SyntaxError: invalid syntax (2006934609.py, line 1)

In [66]:
x_train = df_standardized.drop('y', axis=1)

In [67]:
x_train.head()

,age,default,balance,housing,loan,day,month,duration,number of contacts,#days gap b/w contact,...,marital_married,marital_single,education_secondary,education_tertiary,education_unknown,contact_telephone,contact_unknown,poutcome_other,poutcome_success,poutcome_unknown
0,1.606965,0,0.256419,1,0,5,may,0.011016,-0.569351,-0.411009,...,1,0,0,1,0,0,1,0,0,1
1,0.288529,0,-0.437895,1,0,5,may,-0.416127,-0.569351,-0.411009,...,0,1,1,0,0,0,1,0,0,1
2,-0.747384,0,-0.446762,1,1,5,may,-0.707361,-0.569351,-0.411009,...,1,0,1,0,0,0,1,0,0,1
3,0.571051,0,0.047205,1,0,5,may,-0.645231,-0.569351,-0.411009,...,1,0,0,0,1,0,1,0,0,1
4,-0.747384,0,-0.447091,0,0,5,may,-0.233620,-0.569351,-0.411009,...,0,1,0,0,1,0,1,0,0,1


In [68]:
y_train = df_standardized['y']

In [69]:
y_train

0         no
1         no
2         no
3         no
4         no
        ... 
45206    yes
45207    yes
45208    yes
45209     no
45210     no
Name: y, Length: 45211, dtype: object

In [72]:
y_train.replace({'no':0, 'yes':1}, inplace=True)

In [73]:
y_train

0        0
1        0
2        0
3        0
4        0
        ..
45206    1
45207    1
45208    1
45209    0
45210    0
Name: y, Length: 45211, dtype: int64